In [2]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from topogram_client import TopogramAPIClient

from csv import DictReader
import os
import json
import pymongo
#import musicbrainzngs
import arrow
from datetime import datetime
from pymongo import MongoClient
from collections import defaultdict
from dateutil import parser
from datetime import timedelta
import pickle
import time
from slugify import slugify
from fuzzywuzzy import fuzz
import urllib
import math

##SET PATH TO SAVE FILES:
savpath="./bands_dicts/"


In [3]:
#a function to have a range of colors for nodes/edges 
import colorsys
def get_N_HexCol(N=20):

    HSV_tuples = [(x*1.0/N, 0.5, 0.5) for x in xrange(N)]
    hex_out = []
    for rgb in HSV_tuples:
        rgb = map(lambda x: int(x*255),colorsys.hsv_to_rgb(*rgb))
        hex_out.append("".join(map(lambda x: chr(x).encode('hex'),rgb)))
    return hex_out

In [4]:
def create_topogram(title, nodes, edges):

    print "Creating topogram '%s'"%title

    try :
        r = topogram.create_topogram(title)
    except ValueError:
        print '> Topogram already exists'
        r = topogram.get_topogram_by_name(title)
        
    print r
    topogram_ID = r["data"]["_id"]
    print "topogram ID : %s"%topogram_ID

    # get and backup existing nodes and edges
    existing_nodes = topogram.get_nodes(topogram_ID)["data"]
    existing_edges = topogram.get_edges(topogram_ID)["data"]

    # clear existing graph
    if len(existing_nodes) and  len(existing_nodes) != len(nodes):
        topogram.delete_nodes([n["_id"] for n in existing_nodes])
        print "%s nodes deleted"%len(existing_nodes)
        r = topogram.create_nodes(topogram_ID, nodes)
        print "%s nodes created."%len(r["data"])
    elif len(existing_nodes) and  len(existing_nodes) == len(nodes):
        print "nodes already created, SKIPPED"
    else:
        r = topogram.create_nodes(topogram_ID, nodes)
        print "%s nodes created."%len(r["data"])
        
    if len(existing_edges) and  len(existing_edges) != len(edges):
        topogram.delete_edges([n["_id"] for n in existing_edges])
        print "%s edges deleted"%len(existing_edges)
        r = topogram.create_edges(topogram_ID, edges)
        print "%s edges created."%len(r["data"])
    elif len(existing_edges) and  len(existing_edges) == len(edges):
        print "edges already created,SKIPPED"
    else: 
        r = topogram.create_edges(topogram_ID, edges)
        print "%s edges created."%len(r["data"])

    #r = topogram.create_nodes(topogram_ID, nodes)
    #print r
    
    

    #r=topogram.makePublic(topogram_ID)
    print "done. Topogram has been updated. Check it at %s/topograms/%s"%(TOPOGRAM_URL, topogram_ID)



In [13]:
artist={}
artist["name"]="steve-aoki"
artist["mbid"]="a39acc4a-985e-4173-a9ed-f325f2d3bc1c"

with open("../../4_miner/tour_miner/"+slugify(artist["name"]+artist["mbid"])+".obj", 'rb') as handle:
          tempL= pickle.load( handle)
print tempL

{1: {'tourLength': 9, 'tourdist': 8537.241216520675, 'tourYear': 2008, 'tourdistOpt': 9340.712656690901}, 2: {'tourLength': 3, 'tourdist': 3897.598127259958, 'tourYear': 2008, 'tourdistOpt': 3897.598127259958}, 3: {'tourLength': 291, 'tourdist': 925631.7134221339, 'tourYear': 2009, 'tourdistOpt': 89218.09486013964}, 4: {'tourLength': 131, 'tourdist': 356680.719377711, 'tourYear': 2011, 'tourdistOpt': 69523.71811121632}, 'name': u'Steve Aoki', 6: {'tourLength': 355, 'tourdist': 914263.7746566971, 'tourYear': 2012, 'tourdistOpt': 100188.48034786052}, 7: {'tourLength': 188, 'tourdist': 546725.6589293461, 'tourYear': 2014, 'tourdistOpt': 78159.49849836492}, 8: {'tourLength': 102, 'tourdist': 278360.29618725023, 'tourYear': 2015, 'tourdistOpt': 65190.75458792534}, 9: {'tourLength': 235, 'tourdist': 769053.0295231593, 'tourYear': 2016, 'tourdistOpt': 86930.7992358514}, 'totalArtistOptTourDist': 516304.3627186067, 'OptArtistPercent': 86.5401284841919, 'totalArtistTourDist': 3835878.835189679,

In [137]:
###HERE WE BUILD A LABEL OWNERSHIP CRUSHER FACTORIZED
with open("../../4_miner/labels_aliases_TOTAL.obj", 'rb') as handle:
          tempRelAll= pickle.load( handle)
def find_owning_label(label):
    
    labrels=[]
    if "label-relation-list" in tempRelAll[label]["label"] :

        for relat in tempRelAll[label]["label"]["label-relation-list"]:
            if "direction" in relat and relat["direction"] == "backward":
                print tempRelAll[label]["label"]["name"],"OWNED BY",relat["target"]," ",tempRelAll[relat["target"]]["label"]["name"]
                labrels.append(relat["target"])
                rum_and_whisky=""
                while rum_and_whisky != "SELF": 
                    rum_and_whisky=find_owning_label(relat["target"])
                print "ENDED LOOP"
                labrels.remove(relat["target"])
                print "LABRELS",labrels
                #if labrels[0]==relat["target"]:
                 #   break
                
            else:
                pass
            
        
        if not labrels :
            print "TOP OF THE TREE FOR ",tempRelAll[label]["label"]["name"]
            return "SELF"
        else:
            return labrels
                


artist={}
artist["name"]="Steve Aoki"
artist["mbid"]="a39acc4a-985e-4173-a9ed-f325f2d3bc1c"

with open("../../4_miner/release_mbnz/"+artist["name"]+artist["mbid"]+".obj", 'rb') as handle:
          tempRelz= pickle.load( handle)
#print tempRel["release-list"][0]['label-info-list'][0]['label']['id']
#print tempRel["release-list"][0]['release-event-list'][0]["date"]




for release in tempRelz["release-list"]:
    print"---------------------------"
    print release['label-info-list'][0]['label']['id']
    print release['label-info-list'][0]['label']['name']
    print release['release-event-list'][0]["date"]
    
    #print tempRelAll[release['label-info-list'][0]['label']['id']]["label"]
    try:
        result_lab=find_owning_label(release['label-info-list'][0]['label']['id'])
        print "REZLAB",result_lab 
        print"XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
    except KeyError as e:
        #print e
        pass

---------------------------
6c690937-c6c3-4c4e-9d3e-c62efa989333
Ultra Records
2012-07-10
Ultra Records OWNED BY f0d2ccb8-a269-491c-bbe3-08bdcd3251b7   Ultra Records, Inc.
TOP OF THE TREE FOR  Ultra Records, Inc.
ENDED LOOP
LABRELS []
TOP OF THE TREE FOR  Ultra Records
REZLAB SELF
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
---------------------------
44510f75-af0c-49b5-ab17-179f38452781
Ministry of Sound (Australia)
2010-05-14
Ministry of Sound (Australia) OWNED BY c48b8ed2-46c5-4e51-9bea-5a37fcd0053b   Ministry of Sound
Ministry of Sound OWNED BY 136021ca-a54d-4451-99c9-6ecf52121b86   Ministry of Sound Recordings Ltd.
Ministry of Sound Recordings Ltd. OWNED BY 9a40374c-c95b-47bc-b353-b2b9728848d3   Sony Music Entertainment UK Limited
Sony Music Entertainment UK Limited OWNED BY b54769e4-b75e-4f60-884a-c4714687bb4c   Sony Music Entertainment
Sony Music Entertainment OWNED BY a57fa9c5-f488-4bae-9eb6-41b734227e90   Sony Corporation of America
TOP OF THE TREE FOR  Sony Corporation of America
ENDED

In [5]:
# setup mongo
client = MongoClient()
db = client["bandstour"]
bandsintown =  db["minedArtists"]

# credentials
TOPOGRAM_URL = "http://localhost:3000"
USER = "gregory.bahde@laposte.net"
PASSWORD = "matrix"

# data
#title = "Foo Fighters bandstour 0.1"
my_nodesdict = {}
my_nodes= []
my_edges = []
my_edgesdict = {}
my_nodesdictGLOBAL={}

#LIVENATION CATALOGUE
liste_salles_LN = json.load(open('./JSON_LIVENATION_VENUE.json'))

liste_salle_parsee_LN={}
for salle in liste_salles_LN:
    if liste_salles_LN[salle]["type"]=="venue":
 
            liste_salle_parsee_LN[slugify(liste_salles_LN[salle]["title"])]={"lat":liste_salles_LN[salle]["position"]["lat"],"lng":liste_salles_LN[salle]["position"]["lng"]}
#print liste_salle_parsee_LN


# date parsing
DATETIME_FORMAT = '%Y-%m-%dT%H:%M:%S'
DATETIME_FORMAT_FOR_MSBNZ ='%Y-%m-%d'

#retrieve trip coords for each artist
dict_coords={}
colors_for_nodes_and_edges = get_N_HexCol()

for artist in db.minedArtists.find({"totalKm": {"$gte": 200000}}) :
 title = artist["name"]   
 if os.path.exists(savpath+slugify(artist["name"]+str(artist["mbid"]))+".obj")==True:
        my_nodesdict = {}
        my_nodesdictList=[]
        print "SKIPPED PROCESSING BUT TAKING INTO ACCOUNT ",artist["name"]
        with open(savpath+slugify(artist["name"]+str(artist["mbid"]))+".obj", 'rb') as handle:
          tempL= pickle.load( handle)
    #print 
        tit=tempL[0].encode("utf-8")
        #edgestop=temp[2]
        my_nodesdictList=tempL[1]
        for node in my_nodesdictList:
            my_nodesdict[node["data"]["id"]]=node

        #print my_nodes
        #pass
 else:
  my_nodesdict = {}
  my_nodes= []
  my_edges = []
  my_nodesdictList=[]
  my_edgesdict = {}
    #colors_for_nodes_and_edges = get_N_HexCol()
    
  print artist["name"]
   #print artist
  dates_coords = []
  dates = artist.get("gigs")
  tours = artist.get("tours")    
  singlegigs = artist.get("singleGigs")
  dateini=0
  touring=0
  tour_indice=0
  tour_counter_for_CO2=1
  tour_counter_for_CO2_lock=0
  tmpOLDgig ={}
  if os.path.exists("../../4_miner/tour_miner/"+slugify(artist["name"]+str(artist["mbid"]))+".obj") == True:
    with open("../../4_miner/tour_miner/"+slugify(artist["name"]+str(artist["mbid"]))+".obj", 'rb') as handle:
                    tempCO2= pickle.load( handle)
    #print tempCO2
    #print tours
    for tour in tours:
        
        ##WE NEED TO MATCH TOUR INDICES OF CO2 SPECS FILES...
     if tour_counter_for_CO2_lock == 1:
        tour_counter_for_CO2+=1
     tour_counter_for_CO2_lock=0
     tour_color = "#"+colors_for_nodes_and_edges[tour_indice % 20]
     #print tour_color
     #print tour
     gig_indice =0
     dates = tour.get("gigs")
     for date in dates:
        #print date.get("datetime")
        venue = date.get("venue")
        latitude = venue.get("latitude")
        longitude = venue.get("longitude")
        #print venue
        ###concat key
        idd= slugify(venue["name"])+str(int(venue["latitude"]))+str(int(venue["longitude"]))
        
        if idd not in my_nodesdict:
            
            node = {
                    "id" : idd,
                    "name" : slugify(venue["name"]),
                    "lat" : float(venue["latitude"]),
                    "lng" : float(venue["longitude"]),
                    "weight" : float(1),
                    "start" : date.get("datetime").isoformat(),
                    "end" : (date.get("datetime")+timedelta(days=1)).isoformat(),
                    "color" : tour_color,
                    "notes" : "name : "+slugify(venue["name"])+"  \nlat : "+str(float(venue["latitude"]))+"  \nlng :"+str(float(venue["longitude"]))+"  \n weight :"+str(1)+"  \nstart : "+str(date.get("datetime").isoformat())+"  \nend : "+str((date.get("datetime")+timedelta(days=1)).isoformat())+"  \ncolor :"+str(tour_color)+"  \n  <a href=\"https://google.com/search?q="+slugify(venue["name"])+"\" target=\"_blank\">"+slugify(venue["name"])+"</a>"
                #+"  \n[search venue named "+slugify(venue["name"])+" on google](https://google.com/search?q="+slugify(venue["name"])+")"
                    }
        else:
                
                node = {
                    "id" : idd,
                    "name" : slugify(venue["name"]),
                    "lat" : float(venue["latitude"]),
                    "lng" : float(venue["longitude"]),
                    "weight" : math.sqrt(float(int(round(float(my_nodesdict[idd]["data"].get("weight"))**2))+1)) ,
                    "start" : my_nodesdict[idd]["data"].get("start"),
                    "end" : (date.get("datetime")+timedelta(days=1)).isoformat(),
                    "color" : tour_color,
                    "notes" : "name : "+slugify(venue["name"])+"  \nlat : "+str(float(venue["latitude"]))+"  \nlng :"+str(float(venue["longitude"]))+"  \n weight :"+str((float(my_nodesdict[idd]["data"].get("weight")))**2+1)+"  \nstart : "+str(my_nodesdict[idd]["data"].get("start"))+"  \nend : "+(date.get("datetime")+timedelta(days=1)).isoformat()+"  \ncolor :"+tour_color+"  \n  <a href=\"https://google.com/search?q="+slugify(venue["name"])+"\" target=\"_blank\"> "+slugify(venue["name"])+" </a>"                    #+"  \n[search venue named "+slugify(venue["name"])+" on google](https://google.com/search?q="+slugify(venue["name"])+")"
                    
                    }
        if dateini == 0:
                dateini =1
        else:
                #print my_nodes[-1]["data"]["id"]
                #print dates[gig_indice-1]["distanceToNextGig"]
                
                
                
                notesE=""
                notesE+="distance : "
                try: 
                    notesE+=str(dates[gig_indice]["distanceToNextGig"])
                except KeyError:
                    notesE+="dates[gig_indice][distanceToNextGig] NOT AVAILABLE"
                notesE+=" km  \n"
                notesE+="source : "
                notesE+=str(my_nodes[-1]["data"]["id"])
                notesE+="  target : "
                notesE+=str(node["id"])
                notesE+="  \ndatesource : "
                notesE+=dates[gig_indice -1]["datetime"].isoformat()
                notesE+="  \ndatetarget : "
                notesE+=dates[gig_indice]["datetime"].isoformat()
                notesE+="  \ngroup: "
                notesE+=str(int(tour_indice)+1)
                #notesE+=
                
                
                #print gig_indice
                edge = {
                    "source" : my_nodes[-1]["data"]["id"],
                    "target" : node["id"],
                    "color" :  tour_color,
                    "notes" : notesE,
                    #"weight" : float(e["weight"]),
                    "start" : dates[gig_indice -1]["datetime"].isoformat(),
                    "end" : dates[gig_indice]["datetime"].isoformat(),
                    "group" : str(tour_counter_for_CO2)
                    
                }
                #print edge
                #ADDING EDGES GROUPS/CO2 HERE
                
                if tour["nbGigs"] > 2:
                    try:
                        if tempCO2[tour_counter_for_CO2]['tourdist'] > 1.25*tempCO2[tour_counter_for_CO2]['tourdistOpt']:
                            
                            edge["group"]+=" DASHED2"
                            edge["notes"]+="  \n tournée grandement optimisable  \nDistance parcourue pendant le tour: "+str(tempCO2[tour_counter_for_CO2]['tourdist'])+" km  \n"
                            edge["notes"]+="Distance recalculée: "+str(tempCO2[tour_counter_for_CO2]['tourdistOpt'])+" km  \n"
                            try:
                                edge["notes"]+="Taux d'optimisation "+str((tempCO2[tour_counter_for_CO2]['tourdist']-tempCO2[tour_counter_for_CO2]['tourdistOpt'])/tempCO2[tour_counter_for_CO2]['tourdist'])+" %"
                            except:
                                print "OUPS 2"
                        elif (tempCO2[tour_counter_for_CO2]['tourdist'] < 1.25*tempCO2[tour_counter_for_CO2]['tourdistOpt']) and (tempCO2[tour_counter_for_CO2]['tourdist'] > 1.1*tempCO2[tour_counter_for_CO2]['tourdistOpt']):
                            
                            edge["group"]+=" DASHED1"
                            edge["notes"]+="  \n  tournée optimisable  \nDistance parcourue pendant le tour: "+str(tempCO2[tour_counter_for_CO2]['tourdist'])+" km  \n"
                            edge["notes"]+="Distance recalculée: "+str(tempCO2[tour_counter_for_CO2]['tourdistOpt'])+" km  \n"
                            try:
                                edge["notes"]+="Taux d'optimisation "+str((tempCO2[tour_counter_for_CO2]['tourdist']-tempCO2[tour_counter_for_CO2]['tourdistOpt'])/tempCO2[tour_counter_for_CO2]['tourdist'])+" %"
                            except:
                                print "OUPS 1"
                        elif tempCO2[tour_counter_for_CO2]['tourdistOpt'] > 1.25*tempCO2[tour_counter_for_CO2]['tourdist']:
                            
                            edge["group"]+=" DASHED-2"
                            edge["notes"]+="  \n tournée déjà grandement optimisée  \nDistance parcourue pendant le tour: "+str(tempCO2[tour_counter_for_CO2]['tourdist'])+" km  \n"
                            edge["notes"]+="Distance recalculée: "+str(tempCO2[tour_counter_for_CO2]['tourdistOpt'])+" km  \n"
                            try:
                                edge["notes"]+="Taux d'optimisation "+str((tempCO2[tour_counter_for_CO2]['tourdist']-tempCO2[tour_counter_for_CO2]['tourdistOpt'])/tempCO2[tour_counter_for_CO2]['tourdist'])+" %"
                            except:
                                print "OUPS -2"
                        elif tempCO2[tour_counter_for_CO2]['tourdistOpt'] < 1.25*tempCO2[tour_counter_for_CO2]['tourdist'] and tempCO2[tour_counter_for_CO2]['tourdistOpt'] > 1.1*tempCO2[tour_counter_for_CO2]['tourdist']:
                            
                            edge["group"]+=" DASHED-1"
                            edge["notes"]+="  \n  tournée déjà optimisée  \nDistance parcourue pendant le tour: "+str(tempCO2[tour_counter_for_CO2]['tourdist'])+" km  \n"
                            edge["notes"]+="Distance recalculée: "+str(tempCO2[tour_counter_for_CO2]['tourdistOpt'])+" km  \n"
                            try:
                                edge["notes"]+="Taux d'optimisation "+str((tempCO2[tour_counter_for_CO2]['tourdist']-tempCO2[tour_counter_for_CO2]['tourdistOpt'])/tempCO2[tour_counter_for_CO2]['tourdist'])+" %"
                            except:
                                print "OUPS -1"
                        
                        else:

                            edge["notes"]+="  \nDistance parcourue pendant le tour: "+str(tempCO2[tour_counter_for_CO2]['tourdist'])+" km  \n"
                        tour_counter_for_CO2_lock=1
                    except KeyError as e:
                        print e
                else:
                    #print "TOUR NON COMPTE ", tour_indice, "//",tour_counter_for_CO2
                    continue
                    
                    
                    
                my_edges.append({ "data" : edge })
        my_nodes.append({ "data" : node })   
        my_nodesdict[node["id"]]= { "data" : node }
        gig_indice+=1
    #print my_nodes
    #print "<=============================================>"
    #print my_nodesdict
    #print "<=============================================>"
    #print my_edges
     tour_indice +=1
        #print "coords",latitude,"/",longitude
    for key, value in my_nodesdict.iteritems():
            my_nodesdictList.append(value)
    
    #ADDING LIVENATION DETECTOR

    for node in my_nodesdictList:
        for salle in liste_salle_parsee_LN:
            if fuzz.partial_ratio( node['data']['name'], salle)>=80:
                #print "ONE HEERE"
                if round(node['data']['lat'],2)==round(liste_salle_parsee_LN[salle]["lat"],2) and round(node['data']['lng'],2)==round(liste_salle_parsee_LN[salle]["lng"],2) :
                    #print "CONFIRMED LIVENATION!"
                    node['data']['color']="#ff0000"
                    node['data']["notes"]+="  \nLivenation venue"
        #if liste_salle_parsee_LN
        
        
    #ADDING EDGES WEIGHT
    terp= map(lambda x : str(sorted([x["data"]["source"],x["data"]["target"]])),my_edges)
    
    #print terp
    from collections import Counter
    c= Counter()
    for ter in terp:
        c[ter]+=1
    #print c
    for edge in my_edges:
        if str(sorted([edge["data"]["source"],edge["data"]["target"]])) not in terp:
            print "BUG!!!!"
        else:
            if edge["data"]["source"] == edge["data"]["target"]:
                #print c[str(sorted([edge["data"]["source"],edge["data"]["target"]]))]
                edge["data"]["notes"]+="  \ncounted "+str(c[str(sorted([edge["data"]["source"],edge["data"]["target"]]))])+"  times"
                edge["data"]["weight"]= float(1)
                #print "a"
            else:
                edge["data"]["notes"]+="  \ncounted "+str(c[str(sorted([edge["data"]["source"],edge["data"]["target"]]))])+"  times"
                edge["data"]["weight"]= float(c[str(sorted([edge["data"]["source"],edge["data"]["target"]]))])
                #print "b"
    #print my_edges

    
    
    
    

    titredugraphe =artist["name"]+"/BETA_0.9"+"  \nDistance totale parcourue par l'artiste: "+str(tempCO2["totalArtistTourDist"])+" km"
    titredugraphe += "  \nDistance calculee par Concorde: "+str(tempCO2["totalArtistOptTourDist"])+" km"
    if  "OptArtistPercent" in tempCO2 :
         rateOpt=0
         rateOpt=float(tempCO2["OptArtistPercent"])
         titredugraphe+="  \nPourcentage d'optimisation global calculee:"+str(rateOpt)+" %"
         if rateOpt > 10: 
             titredugraphe+="  \nMARGE D'OPTIMISATION IMPORTANTE"
         elif rateOpt < -10:
              titredugraphe+="  \nTOURNEE DEJA OPTIMISEE"
         else: 
             titredugraphe+="  \nGLOBALEMENT IDENTIQUE"
         
         
    with open(savpath+slugify(artist["name"]+str(artist["mbid"]))+".obj", 'wb') as handle:
                            pickle.dump([titredugraphe,my_nodesdictList,my_edges], handle, protocol=pickle.HIGHEST_PROTOCOL)    
            
    ###CREATE NODE GLOBAL DICT
           
 for key, value in my_nodesdict.iteritems():
            #print "key",key,"/////",value
            if key in my_nodesdictGLOBAL:
                #print "TROUVE"
                #print "key",key,"/////",value
                #print "GLOBAL",my_nodesdictGLOBAL[key]
                my_nodesdictGLOBAL[key]['data']["weight"]=math.sqrt(my_nodesdictGLOBAL[key]['data']["weight"]**2+my_nodesdict[key]['data']["weight"]**2)
                if time.strptime(my_nodesdictGLOBAL[key]['data']["start"],DATETIME_FORMAT)>time.strptime(my_nodesdict[key]['data']["start"],DATETIME_FORMAT):
                    my_nodesdictGLOBAL[key]['data']["start"]=my_nodesdict[key]['data']["start"]
                if time.strptime(my_nodesdictGLOBAL[key]['data']["end"],DATETIME_FORMAT)<time.strptime(my_nodesdict[key]['data']["end"],DATETIME_FORMAT):
                    my_nodesdictGLOBAL[key]['data']["end"]=my_nodesdict[key]['data']["end"]
                my_nodesdictGLOBAL[key]['data']["notes"]+="  \n"+slugify(artist["name"])
                my_nodesdictGLOBAL[key]['data']["count"]+=int(1)
                #print "GLOBAL2",my_nodesdictGLOBAL[key]
            else:
                my_nodesdictGLOBAL[key]=value
                #print my_nodesdictGLOBAL[key]
                my_nodesdictGLOBAL[key]['data']["notes"]="  \n List of the bands that played in this venue:  \n"+slugify(artist["name"])
                my_nodesdictGLOBAL[key]['data']["count"]=int(1)
for key in my_nodesdictGLOBAL:
    my_nodesdictGLOBAL[key]['data']["notes"]+="  \n TOTAL: "+str(my_nodesdictGLOBAL[key]['data']["count"])+" Bands"
    
    
with open("./my_nodesdictGLOBAL_TEST.obj", 'wb') as handle:
                            pickle.dump(my_nodesdictGLOBAL, handle, protocol=pickle.HIGHEST_PROTOCOL)

!!!
(həd) p.e.
.38 Special
SKIPPED PROCESSING BUT TAKING INTO ACCOUNT  10 String Symphony
10 Years
SKIPPED PROCESSING BUT TAKING INTO ACCOUNT  10cc
112
SKIPPED PROCESSING BUT TAKING INTO ACCOUNT  12
SKIPPED PROCESSING BUT TAKING INTO ACCOUNT  12 Stones
12th Planet
2 Chainz
2 Many DJ's
2Cellos
3 Doors Down
30
311
3LAU
40 oz. To Freedom
4th Dimension
5 Seconds of Summer
50 Cent
65daysofstatic
68
SKIPPED PROCESSING BUT TAKING INTO ACCOUNT  7 Seconds
SKIPPED PROCESSING BUT TAKING INTO ACCOUNT  7EVENTH TIME DOWN
8 Ball Aitken
SKIPPED PROCESSING BUT TAKING INTO ACCOUNT  A Chorus Line
SKIPPED PROCESSING BUT TAKING INTO ACCOUNT  A Christmas Carol
A Day To Remember
A Guy Called Gerald
A Thousand Horses
SKIPPED PROCESSING BUT TAKING INTO ACCOUNT  A Tribe Called Red
A Wilhelm Scream
A$AP Ferg
A-Trak
SKIPPED PROCESSING BUT TAKING INTO ACCOUNT  A.J. Croce
SKIPPED PROCESSING BUT TAKING INTO ACCOUNT  A.Skillz
SKIPPED PROCESSING BUT TAKING INTO ACCOUNT  A.Z. 
ABBA Mania
SKIPPED PROCESSING BUT TAKING I

KeyboardInterrupt: 

In [ ]:
     with open("./my_nodesdictGLOBAL.obj", 'rb') as handle:
          my_nodesdictGLOBAL= pickle.load( handle)

In [6]:
from itertools import islice

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

print take(5,my_nodesdictGLOBAL.iteritems())

[('ohio-university39-83', {'data': {'count': 1, 'start': '2016-06-10T18:30:00', 'end': '2016-06-11T18:30:00', 'name': 'ohio-university', 'weight': 1.0, 'lat': 39.3268329, 'lng': -83.0063841, 'notes': '  \n List of the bands that played in this venue:  \nled-zepagain  \n TOTAL: 1 Bands', 'id': 'ohio-university39-83', 'color': '#7f3f3f'}}), ('ohio-university39-82', {'data': {'count': 9, 'start': '2008-04-09T19:00:00', 'end': '2015-11-15T19:00:00', 'name': 'ohio-university', 'weight': 3.3784927944829333, 'lat': 39.324921, 'lng': -82.099709, 'notes': '  \n List of the bands that played in this venue:  \nben-folds  \njosh-ritter  \nmachine-gun-kelly  \nray-wylie-hubbard  \nrayland-baxter  \nthe-blind-boys-of-alabama  \ntyler-farr  \nwiz-khalifa  \nzo  \n TOTAL: 9 Bands', 'id': 'ohio-university39-82', 'color': '#5f7f3f'}}), ('tucson-music-hall32-110', {'data': {'count': 35, 'start': '2007-12-17T00:00:00', 'end': '2017-09-15T19:00:00', 'name': 'tucson-music-hall', 'weight': 6.9258925443875565

In [6]:
# credentials
TOPOGRAM_URL = "http://localhost:3000"
USER = "gregory.bahde@laposte.net"
PASSWORD = "matrix"

In [5]:
print len(my_nodesdictGLOBAL)

NameError: name 'my_nodesdictGLOBAL' is not defined

In [7]:
# connect to the topogram instance (pass debug=True params for more info )
topogram = TopogramAPIClient(TOPOGRAM_URL) #, debug=True)

# create a new user
try :
    topogram.create_user(USER, PASSWORD)
except ValueError:
    print "> User has already been created."

# login a new user if needed
resp_user_login = topogram.user_login(USER, PASSWORD)
print resp_user_login

assert(resp_user_login["status"] == "success")
assert(resp_user_login["status_code"] == 200)


ERROR:topogram-client:403 Unauthorized request


> User has already been created.
{u'status': u'success', 'status_code': 200, u'data': {u'authToken': u'G4HUbMjMrqE6D8W5L4SMOoMAG6MHK2kLjeHN4ZeJjbj', u'userId': u'apyw7AfhAGbEHigJw'}}


In [ ]:
###IMPORT ALL THE BANDS GRAPHS

nodestop=[]
edgestop=[]
tit=[]
import glob, os

print os.getcwd()

for subdir, dirs, files in os.walk(savpath): 
 for file in files:
  if file.endswith('.obj') == True:
    #print file
    with open(os.path.join(subdir, file), 'rb') as handle:
          temp= pickle.load( handle)
    #print temp 
    tit=temp[0].encode("utf-8")
    edgestop=temp[2]
    nodestop=temp[1]
    #print edgestop
    #for edge in edgestop:
    #    assert edge["data
    while True:
        try:
    #        print tit
            create_topogram(tit, nodestop, edgestop)
            
        except TypeError:
            continue
        
        break
print 'DONE'
        

ERROR:topogram-client:500 - Error : A topogram with the same name already exists


/home/goonieb/gitrep/bandstour/5_visualizer/topogram-api-clientFOR_TOPO_FROM_BANDSTOUR
Creating topogram 'Mannheim Steamroller Christmas/BETA_0.9  
Distance totale parcourue par l'artiste: 192784.711872 km  
Distance calculee par Concorde: 70683.1134013 km  
Pourcentage d'optimisation global calculee:63.3357268246 %  
MARGE D'OPTIMISATION IMPORTANTE'
> Topogram already exists
{u'status': u'success', 'status_code': 200, u'data': {u'sharedPublic': False, u'title': u"Mannheim Steamroller Christmas/BETA_0.9  \nDistance totale parcourue par l'artiste: 192784.711872 km  \nDistance calculee par Concorde: 70683.1134013 km  \nPourcentage d'optimisation global calculee:63.3357268246 %  \nMARGE D'OPTIMISATION IMPORTANTE", u'userId': u'apyw7AfhAGbEHigJw', u'_id': u'MrK4MiychDYa4zzr3', u'slug': u'mannheim-steamroller-christmasbeta_09-distance-totale-parcourue-par-lartiste-192784711872-km-distance-calculee-par-concorde-706831134013-km-pourcentage-doptimisation-global-calculee633357268246-marge-dopti

ERROR:topogram-client:500 - Error : A topogram with the same name already exists


nodes already created, SKIPPED
edges already created,SKIPPED
done. Topogram has been updated. Check it at http://localhost:3000/topograms/MrK4MiychDYa4zzr3
Creating topogram 'Reign/BETA_0.9  
Distance totale parcourue par l'artiste: 106909.768042 km  
Distance calculee par Concorde: 66099.2059829 km  
Pourcentage d'optimisation global calculee:38.1729030064 %  
MARGE D'OPTIMISATION IMPORTANTE'
> Topogram already exists
{u'status': u'success', 'status_code': 200, u'data': {u'sharedPublic': False, u'title': u"Reign/BETA_0.9  \nDistance totale parcourue par l'artiste: 106909.768042 km  \nDistance calculee par Concorde: 66099.2059829 km  \nPourcentage d'optimisation global calculee:38.1729030064 %  \nMARGE D'OPTIMISATION IMPORTANTE", u'userId': u'apyw7AfhAGbEHigJw', u'_id': u'zJZTbr2GtbMjGGDMB', u'slug': u'reignbeta_09-distance-totale-parcourue-par-lartiste-106909768042-km-distance-calculee-par-concorde-660992059829-km-pourcentage-doptimisation-global-calculee381729030064-marge-doptimisati

ERROR:topogram-client:500 - Error : A topogram with the same name already exists


...

ERROR:topogram-client:500 - Error : A topogram with the same name already exists


nodes already created, SKIPPED
edges already created,SKIPPED
done. Topogram has been updated. Check it at http://localhost:3000/topograms/AyFytozZL44TNZae9
Creating topogram 'The Warlocks/BETA_0.9  
Distance totale parcourue par l'artiste: 110554.138509 km  
Distance calculee par Concorde: 102414.234499 km  
Pourcentage d'optimisation global calculee:7.36282161781 %  
GLOBALEMENT IDENTIQUE'
> Topogram already exists
{u'status': u'success', 'status_code': 200, u'data': {u'sharedPublic': False, u'title': u"The Warlocks/BETA_0.9  \nDistance totale parcourue par l'artiste: 110554.138509 km  \nDistance calculee par Concorde: 102414.234499 km  \nPourcentage d'optimisation global calculee:7.36282161781 %  \nGLOBALEMENT IDENTIQUE", u'userId': u'apyw7AfhAGbEHigJw', u'_id': u'5vLaALRL8qBu88tbE', u'slug': u'the-warlocksbeta_09-distance-totale-parcourue-par-lartiste-110554138509-km-distance-calculee-par-concorde-102414234499-km-pourcentage-doptimisation-global-calculee736282161781-globalement-iden

ERROR:topogram-client:500 - Error : A topogram with the same name already exists


nodes already created, SKIPPED
edges already created,SKIPPED
done. Topogram has been updated. Check it at http://localhost:3000/topograms/5vLaALRL8qBu88tbE
Creating topogram 'Ernesto Ferreyra/BETA_0.9  
Distance totale parcourue par l'artiste: 81214.0114567 km  
Distance calculee par Concorde: 96690.5832891 km  
Pourcentage d'optimisation global calculee:-19.0565292304 %  
TOURNEE DEJA OPTIMISEE'
> Topogram already exists
{u'status': u'success', 'status_code': 200, u'data': {u'sharedPublic': False, u'title': u"Ernesto Ferreyra/BETA_0.9  \nDistance totale parcourue par l'artiste: 81214.0114567 km  \nDistance calculee par Concorde: 96690.5832891 km  \nPourcentage d'optimisation global calculee:-19.0565292304 %  \nTOURNEE DEJA OPTIMISEE", u'userId': u'apyw7AfhAGbEHigJw', u'_id': u'aX6cK5NTeExpznWdS', u'slug': u'ernesto-ferreyrabeta_09-distance-totale-parcourue-par-lartiste-812140114567-km-distance-calculee-par-concorde-966905832891-km-pourcentage-doptimisation-global-calculee-190565292304

ERROR:topogram-client:500 - Error : A topogram with the same name already exists


In [50]:
###IMPORT THE GLOBAL VENUE  GRAPH
my_nodesdictList=[]
nodestopo=[]
edgestopo=[]
with open("./my_nodesdictGLOBAL.obj", 'rb') as handle:
          my_nodesdictGLOBAL= pickle.load( handle)
        
for key, value in my_nodesdictGLOBAL.iteritems():
            my_nodesdictGLOBAL[key]["data"].pop('count')
        
for key, value in my_nodesdictGLOBAL.iteritems():
            my_nodesdictList.append(value)
        
nodestopo=sorted(my_nodesdictList, key= lambda (v) : v['data']['weight'], reverse=True)[:5000]
print len(nodestopo)



title="global venue graph /BETA 0.6"

while True:
        try:
            print nodestopo
            create_topogram(title, nodestopo, edgestopo)
        
        except TypeError:
            continue
        break
        

ERROR:topogram-client:500 - Error : A topogram with the same name already exists
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


5000 nodes deleted
5000 nodes created.


ValueError: No JSON object could be decoded

In [14]:
with open("./bands_dicts/the-real-mckenzies01612542-01ec-4fc4-a9ad-b6a6e33f11d7.obj", 'rb') as handle:
          DEBUG= pickle.load( handle)


In [18]:
print DEBUG[:2]


[u"The Real McKenzies/BETA_0.8  \nDistance totale parcourue par l'artiste: 180691.650535 km  \nDistance calculee par Concorde: 156448.795249 km  \nPourcentage d'optimisation global calculee:13.4166992303 %  \nMARGE D'OPTIMISATION IMPORTANTE", [{'data': {'end': '2015-03-27T20:30:00', 'name': 'the-horseshoe-tavern', 'weight': 1.0, 'color': '#7f663f', 'notes': 'name : the-horseshoe-tavern  \nlat : 43.6490438  \nlng :-79.395913  \n weight :1  \nstart : 2015-03-26T20:30:00  \nend : 2015-03-27T20:30:00  \ncolor :#7f663f  \n  <a href="https://google.com/search?q=the-horseshoe-tavern" target="_blank">the-horseshoe-tavern</a>', 'start': '2015-03-26T20:30:00', 'lat': 43.6490438, 'lng': -79.395913, 'id': 'the-horseshoe-tavern43-79'}}, {'data': {'end': '2012-05-09T20:00:00', 'name': 'the-pyramid-cabaret', 'weight': 1.0, 'color': '#723f7f', 'notes': 'name : the-pyramid-cabaret  \nlat : 49.890735  \nlng :-97.137711  \n weight :1  \nstart : 2012-05-08T20:00:00  \nend : 2012-05-09T20:00:00  \ncolor :#